# Wikipedia scrape

In [4]:
pip freeze

anyio==3.6.1
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
asttokens==2.0.5
attrs==21.4.0
Babel==2.10.1
backcall==0.2.0
beautifulsoup4==4.11.1
bleach==5.0.0
certifi==2022.5.18.1
cffi==1.15.0
charset-normalizer==2.0.12
cycler==0.11.0
debugpy==1.6.0
decorator==5.1.1
defusedxml==0.7.1
entrypoints==0.4
executing==0.8.3
fastjsonschema==2.15.3
fonttools==4.33.3
idna==3.3
imageio==2.19.3
importlib-metadata==4.11.4
importlib-resources==5.7.1
ipykernel==6.13.1
ipython==8.4.0
ipython-genutils==0.2.0
jedi==0.18.1
Jinja2==3.1.2
joblib==1.1.0
json5==0.9.8
jsonschema==4.6.0
jupyter-client==7.3.4
jupyter-core==4.10.0
jupyter-server==1.17.1
jupyterlab==3.4.3
jupyterlab-pygments==0.2.2
jupyterlab-server==2.14.0
kiwisolver==1.4.2
lime==0.2.0.1
MarkupSafe==2.1.1
matplotlib==3.5.2
matplotlib-inline==0.1.3
mistune==0.8.4
nbclassic==0.3.7
nbclient==0.6.4
nbconvert==6.5.0
nbformat==5.4.0
nest-asyncio==1.5.5
networkx==2.8.3
notebook==6.4.12
notebook-shim==0.1.0
numpy==1.22.4
packaging==21.3
pandas==1.4.2
p

In [3]:
import wikipedia
import pywikibot as pw
import pandas as pd

from tqdm import tqdm

In [2]:
pip install wikitextparser>=0.47.5

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
categories = {'Finance':['Finance', 'Valuation (finance)', 'Accounting'],
              'Technology':['Software engineering','Machine learning','Data science','Computer security'],
              'Life Science':['Biology', 'Biotechnology', 'Health sciences'],
              'Business Administration':['Business','Management','Consulting', 'Office administration']}

In [3]:
unique_categories = list(set(categories.keys()))
label_map = {cat:i for i,cat in enumerate(unique_categories)}
label_map

{'Business Administration': 0,
 'Life Science': 1,
 'Technology': 2,
 'Finance': 3}

In [4]:
all_text = {}
site = pw.Site('en','wikipedia')
for category in tqdm(categories.keys()):
    all_text[category] = {}
    for wiki_cat in categories[category]:
        try:
            cat = pw.Category(site,wiki_cat)
        except:
            print(f"Couldn't get cat: {wiki_cat}")
            continue;
        page_names = [x.title() for x in cat.articles()]
        for page_name in page_names:
            try:
                summary = wikipedia.summary(page_name)
            except:
                summary = None
            if summary:
                all_text[category][page_name] = summary

 25%|████████████████████▊                                                              | 1/4 [02:29<07:28, 149.47s/it]WARNING: C:\Users\skarb\Environments\general\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file C:\Users\skarb\Environments\general\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [18:15<00:00, 273.86s/it]


In [5]:
len(all_text)

4

In [6]:
all_text.keys()

dict_keys(['Finance', 'Technology', 'Life Science', 'Business Administration'])

In [7]:
labels = []
cats = []
pages = []
txts = []
for i,cat in enumerate(all_text.keys()):
    for article in all_text[cat].keys():
        labels.append(label_map[cat])
        cats.append(cat)
        pages.append(article)
        txts.append(all_text[cat][article])
df = pd.DataFrame({'category':cats,'page':pages,'text':txts,'label':labels})

In [8]:
df.head()

,category,page,text,label
0,Finance,Portal:Banks,"The human back, also called the dorsum, is the...",3
1,Finance,Finance,Finance is the study of money and assets. It i...,3
2,Finance,Approved Publication Arrangement,With MiFID II directive being in force in Janu...,3
3,Finance,Asset,"In financial accounting, an asset is any resou...",3
4,Finance,Austerity,Austerity is a set of political-economic polic...,3


In [62]:
row1 = df[df['label']==0].iloc[54]
row2 = df[df['label']==1].iloc[1]
row3 = df[df['label']==2].iloc[15]
row4 = df[df['label']==3].iloc[3]

sample = pd.DataFrame([row1,row2,row3,row4])

In [63]:
sample

,category,page,text,label
1226,Business Administration,Hands-on management,Hands-on management is a particular style of m...,0
638,Life Science,Biology,"Geology (from Ancient Greek γῆ (gê) 'earth', ...",1
190,Technology,Site reliability engineering,Site reliability engineering (SRE) is a set of...,2
3,Finance,Asset,"In financial accounting, an asset is any resou...",3


In [9]:
df.shape

(1313, 4)

In [10]:
df.to_csv("./wikipedia_data.csv",index=False)